In [ ]:

# Files to load
#'complete_coins_list.csv'
# latest Master_sentiment_[date].csv
# credentials.txt


'complete_coins_list.csv'

In [1]:
!pip install Telethon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.9/576.9 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for Telethon: filename=Telethon-1.34.0-py3-none-any.whl size=649201 sha256=f9ece369f2357a8cc8351aa15edb6a73dbd494a2d2ec9add040cf6a1be671df6
  Stored in directory: /root/.cache/pip/wheels/d5/28/56/edfb090fbbcd57add61a49dfe194c8eff22b5d4828008f2e62
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=10053c66b08c87685dc2d25fe476100e3a4186b880c28ab54c4110fa3ca5aa52
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built Telethon pyaes


In [2]:
import pandas as pd
import re
import requests
from datetime import datetime, timedelta, date
import numpy as np
import pytz
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, Ridge
from sklearn.pipeline import Pipeline
import matplotlib.dates as mdates

#import ccxt

In [ ]:
#The below sets up a client in Telethon. This is very janky and can have database issues if there is another session open. Sometimes takes a bit of trial and error!

In [10]:
import os

# Delete the session file if it exists
if os.path.exists('anon.session'):
    os.remove('anon.session')


from telethon import TelegramClient
with open('credentials.txt', 'r') as file:
    lines = file.readlines()
    api_id = lines[0].rstrip()  # Remove any trailing newline character
    api_hash = lines[1].rstrip()
client = TelegramClient('apple', api_id, api_hash)
await client.start()

Please enter your phone (or bot token): +447772762162
Please enter the code you received: 22717
Please enter your password: ··········
Signed in successfully as Crypto Sloth; remember to not break the ToS or you will risk an account ban!


In [ ]:
#Not sure if we need this or can just run the below - is very temperamental
from telethon.sync import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest

with open('credentials.txt', 'r') as file:
    lines = file.readlines()
    api_id = lines[0].rstrip()  # Remove any trailing newline character
    api_hash = lines[1].rstrip()


channel_url = 'https://t.me/OuroborosNarratives'
client = TelegramClient('banana', api_id, api_hash)

async with TelegramClient('banana', api_id, api_hash) as client:
    channel_entity = await client.get_entity(channel_url)
    posts = await client(GetHistoryRequest(
        peer=channel_entity,
        limit=100,
        offset_date=None,
        offset_id=0,
        max_id=0,
        min_id=0,
        add_offset=0,
        hash=0))
    print(posts.messages)

In [ ]:
days = 1 # ie for 1 day


import re
import pytz
import pandas as pd
from datetime import datetime, timedelta

cutoff = datetime.now(pytz.utc) - timedelta(days=days)

chats = ['https://t.me/OuroborosNarratives', 'https://t.me/investingbeanstock', 'https://t.me/lobsters_chat', 'https://t.me/cryptonarratives1',
         'https://t.me/joescrypt', 'https://t.me/smolinsanity', 'https://t.me/Noodles_calls',
         'https://t.me/TPLXPodcastInsights', 'https://t.me/alsalpha', 'https://t.me/alsalpha',
         'https://t.me/rydersjournal', 'https://t.me/ELMagoChamber']

df_list = []

with open('credentials.txt', 'r') as file:
    lines = file.readlines()
    api_id = lines[0].rstrip()  # Remove any trailing newline character
    api_hash = lines[1].rstrip()

for chat in chats:
    channel_url = chat
    print(channel_url)

    session_file_path = '/content/custom_path/anon.session'

    #client = TelegramClient('anon', api_id, api_hash)

    data = []
    async with TelegramClient('banana', api_id, api_hash) as client:
        async for message in client.iter_messages(channel_url):
            if message.date > cutoff:
                print(message.sender_id, ':', message.text, message.date)
                data.append([message.sender_id, message.text, message.date, message.id, message.reply_to_msg_id, message.post_author, message.views, message.peer_id.channel_id ])
            else:
                break

    df = pd.DataFrame(data, columns=["message.sender_id", "message.text"," message.date", "message.id", "message.reply_to_msg_id", "message.post_author", "message.views", "message.peer_id.channel_id" ]) # creates a new dataframe

    group_name = re.search(r"https:\/\/t\.me\/(\w+)", chat)
    group = group_name.group(1)
    df['group'] = group

    df_list.append(df)

In [9]:
df = pd.concat([df_list[0], df_list[1], df_list[2], df_list[3], df_list[4], df_list[5], df_list[6], df_list[7], df_list[8], df_list[9], df_list[10], df_list[11]])
df.reset_index(inplace=True, drop=True)

df['Date'] = pd.to_datetime(df[' message.date'])
df.sort_values(by= 'Date', inplace=True, ascending=False)
df.reset_index(inplace=True, drop=True)
df.head()

,message.sender_id,message.text,message.date,message.id,message.reply_to_msg_id,message.post_author,message.views,message.peer_id.channel_id,group,Date
0,5828308118,"Too bad, could do with one now",2024-03-11 21:18:28+00:00,477566,NaN,None,NaN,1242127973,lobsters_chat,2024-03-11 21:18:28+00:00
1,138526437,none active,2024-03-11 20:43:03+00:00,477565,477561.0,None,NaN,1242127973,lobsters_chat,2024-03-11 20:43:03+00:00
2,2010955221,**Ouroboros Research Daily (11 Mar'24)\n**__Pr...,2024-03-11 20:21:16+00:00,17743,NaN,None,None,1611549397,OuroborosNarratives,2024-03-11 20:21:16+00:00
3,180310994,"yeye ima watching them closely, but its in 1-2...",2024-03-11 19:50:48+00:00,477563,477562.0,None,NaN,1242127973,lobsters_chat,2024-03-11 19:50:48+00:00
4,6234419587,Hedgehog in 1-2 months (https://t.me/thehedgeh...,2024-03-11 19:50:28+00:00,477562,477561.0,None,NaN,1242127973,lobsters_chat,2024-03-11 19:50:28+00:00


In [ ]:
#save to csv
#df.to_csv('TG_7days_150224.csv')

In [ ]:
#Import here if have to use telethon

#df = pd.read_csv('TG_750days_070424.csv')

In [ ]:
import requests
import csv

def fetch_cryptocurrencies(per_page=250):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    page = 1
    all_cryptos = []

    while True:
        params = {
            'vs_currency': 'usd',
            'order': 'market_cap_desc',
            'per_page': per_page,
            'page': page
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            break
        data = response.json()
        if not data:
            break
        all_cryptos.extend(data)
        page += 1

    return all_cryptos

def save_to_csv_using_dataframe(cryptos, filename):
    df = pd.DataFrame(cryptos)

    df = df[['id', 'name', 'symbol', 'market_cap', 'total_volume']]
    df.columns = ['ID', 'Name', 'Ticker', 'Market Cap', '24h Volume']

    df.to_csv(filename, index=False)

In [ ]:
#Keeping those with >$500k ADV

try:
    cryptocurrencies_list = 'complete_coins_list.csv'
except:
    data = fetch_cryptocurrencies()
    if data:
        save_to_csv_using_dataframe(data, 'complete_coins_list.csv')

cryptocurrencies_list = 'complete_coins_list.csv'
coins_list = pd.read_csv(cryptocurrencies_list)
coins_list = coins_list[coins_list['24h Volume'] > 500000]
crypto_symbols_set = set(coins_list['Ticker'].str.upper()) # faster lookup

**Huggingface**

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)
specific_model = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly 

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
specific_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", return_all_scores=True)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificat

In [ ]:
def calculate_sentiment(text):
    neg = -10
    neu = 0
    pos = 10

    try:
      detailed_scores = specific_model(text)
      overall_score = (neg * detailed_scores[0][0]['score']) + (neu * detailed_scores[0][1]['score']) + (pos * detailed_scores[0][2]['score'])
      return overall_score
    except:
      None

In [ ]:
strd = 'STRD looks good here on the pullback'
calculate_sentiment(strd)

9.353589313104749

**Extracting tickers from existing comments and replies**

In [ ]:
#Extracting >2 upper case letters in string
def extract_tickers(text, crypto_symbols):
    if pd.isna(text):
        return []
    uppercase_words = re.findall(r'\b[A-Z]{2,}\b', text)
    return [ticker for ticker in uppercase_words if ticker in crypto_symbols]

In [ ]:
# extracting tickers from comments
df['extracted_coins'] = df['message.text'].apply(
    lambda text: extract_tickers(text, crypto_symbols_set)
)

#set up dict to track the reply to mesage ID and tickers mentioned
reply_to_coins_mapping = {}

# fill reply_to_coins_mapping with correct extracted coins
for index, row in df.iterrows():
  reply_to_coins_mapping[row['message.id']] = row['extracted_coins']

In [ ]:
def inherited_coins(row):
  coins = row['extracted_coins']

  # Ensure coins is a list
  if not isinstance(coins, list):
      coins = [coins] if coins is not None else []

  reply_to_id = row['message.reply_to_msg_id']

  if pd.notnull(reply_to_id):

#What does the [] do in .get(reply_to_id, [])?
      reply_to_coins = reply_to_coins_mapping.get(reply_to_id, [])
      if reply_to_coins is not None:
          coins.extend(reply_to_coins)  # Use extend for lists

  return list(set(coins))


# Apply the function to populate the dataframe with inherited coin mentions
df['coin_mentions'] = df.apply(inherited_coins, axis=1)

In [ ]:
# Filter for entries with coin refs and calculate sentiment score for each comment
df = df[df['coin_mentions'].apply(lambda x: len(x) > 0)]
df['sentiment_score'] = df['message.text'].apply(calculate_sentiment)

df.to_csv('HFSentiment_latest.csv')

In [ ]:
df.head()


**Updating the sentiment dataframe for prices that day**

In [ ]:
master = pd.read_csv('Master_sentiment.csv')
df = pd.read_csv('HFSentiment_latest.csv')
#The below probably needs work re the index
def merge_to_master(df, master):
    new_df = pd.concat([df, master], axis = 0)# sort=True)
    new_df = new_df.drop_duplicates()
    new_df.reset_index(drop=True)
    return new_df



In [ ]:
df = merge_to_master(df, master)
df.to_csv('Master_sentiment_new.csv')

In [ ]:
df.shape

(29359, 19)